<a href="https://colab.research.google.com/github/RiccardoElisei/Tesi-IDS/blob/main/Anomaly%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob

In [ ]:
path = "drive/MyDrive/MachineLearningCVE/"
files = [file for file in glob.glob(path + "**/*.csv", recursive=True)]

In [ ]:
[print(f) for f in files]

drive/MyDrive/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
drive/MyDrive/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
drive/MyDrive/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv
drive/MyDrive/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
drive/MyDrive/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
drive/MyDrive/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
drive/MyDrive/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv
drive/MyDrive/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv


[None, None, None, None, None, None, None, None]

In [ ]:
dataset = [pd.read_csv(f) for f in files]

In [ ]:
for d in dataset:
    print(d.shape)

In [ ]:
dataset = pd.concat([d for d in dataset]).drop_duplicates(keep=False)
dataset.reset_index(drop=True, inplace = True)

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.head()

In [ ]:
print(dataset[' Label'].unique())
len(dataset[' Label'].unique())

In [ ]:
label_names = dataset[' Label'].unique()

import re

label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]

label_names, len(label_names)

In [ ]:
labels = dataset[' Label'].unique()

for i in range(0,len(label_names)):
    dataset[' Label'] = dataset[' Label'].replace({labels[i] : label_names[i]})

dataset[' Label'].unique()

In [ ]:
dataset.isnull().values.any()

In [ ]:
[col for col in dataset if dataset[col].isnull().values.any()]

In [ ]:
dataset['Flow Bytes/s'].isnull().sum()

In [ ]:
before = dataset.shape
dataset.dropna(inplace=True)
after = dataset.shape
before[0] - after[0]

In [ ]:
dataset.isnull().any().any()

In [ ]:
import numpy as np

In [ ]:
labl = dataset[' Label']
dataset = dataset.loc[:, dataset.columns != ' Label'].astype('float64')

In [ ]:
np.all(np.isfinite(dataset))

In [ ]:
nonfinite = [col for col in dataset if not np.all(np.isfinite(dataset[col]))]
nonfinite

In [ ]:
finite = np.isfinite(dataset['Flow Bytes/s']).sum()

dataset.shape[0] - finite

In [ ]:
finite = np.isfinite(dataset[' Flow Packets/s']).sum()

dataset.shape[0] - finite

In [ ]:
dataset = dataset.replace([np.inf, -np.inf], np.nan)

In [ ]:
dataset = dataset.merge(labl, how='outer', left_index=True, right_index=True)

In [ ]:
dataset.dropna(inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
conteggio_per_label = dataset[' Label'].value_counts()

for label, conteggio in conteggio_per_label.items():
    print(f"Numero di flussi per il label '{label}': {conteggio}")


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
np.random.seed(42)
rand_perm = np.random.permutation(dataset.shape[0])

In [ ]:
feature_cols = dataset.columns[:-1]

dataset_subset = dataset.iloc[rand_perm[:9975], :]

web_attack_xss_rows = dataset[dataset[' Label'] == 'Web_Attack_XSS'].sample(n=8)
dataset_subset = pd.concat([dataset_subset, web_attack_xss_rows], ignore_index=True)

infiltration_rows = dataset[dataset[' Label'] == 'Infiltration'].sample(n=7)
dataset_subset = pd.concat([dataset_subset, infiltration_rows], ignore_index=True)

web_attack_sql_injection_rows = dataset[dataset[' Label'] == 'Web_Attack_Sql_Injection'].sample(n=6)
dataset_subset = pd.concat([dataset_subset, web_attack_sql_injection_rows], ignore_index=True)

heartbleed_rows = dataset[dataset[' Label'] == 'Heartbleed'].sample(n=4)
dataset_subset = pd.concat([dataset_subset, heartbleed_rows], ignore_index=True)

dataset_subset.to_csv('dataset_subset.csv', index=False)

conteggio_per_label = dataset_subset[' Label'].value_counts()

for label, conteggio in conteggio_per_label.items():
   print(f"Numero di flussi per il label '{label}': {conteggio}")

In [ ]:
feature_cols = dataset.columns[:-1]
dataset_subset = dataset.iloc[rand_perm[:10000],:]
dataset_subset

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = dataset_subset[feature_cols]
Y = dataset_subset[' Label']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=None, random_state=42)

clf = RandomForestClassifier()

clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)
accuracy_rf = accuracy_score(Y_test, Y_pred)

feature_importance = clf.feature_importances_

feature_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})

feature_df = feature_df.sort_values(by='Importance', ascending=False)

weights_rf = {feature: weight for weight, feature in enumerate(feature_df['Feature'], start=1)}

plt.figure(figsize=(15, 8))
plt.bar(feature_df['Feature'][:50], feature_df['Importance'][:50], color='skyblue')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature RandomForsetClassifier')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(feature_df.head(50))

print(accuracy_rf)

In [ ]:
# Seleziona le prime 10/20/30 feature dal classificatore RandomForest
top_features_rf = feature_df['Feature'][:30].tolist()

selected_features_rf = [' Label'] + top_features_rf

new_dataset_rf = dataset_subset[selected_features_rf]

print(new_dataset_rf.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss, roc_auc_score

X_rf = new_dataset_rf.drop(' Label', axis=1)
y_rf = new_dataset_rf[' Label']

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=None, random_state=42)

model_rf = RandomForestClassifier()

param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_rf, param_grid, random_state=42).fit(X_rf, y_rf)
best_rf_params = search.best_params_

print("Migliori parametri per RandomForest:", best_rf_params)

best_model_rf = RandomForestClassifier(**best_rf_params)
best_model_rf.fit(X_rf, y_rf)

predictions_rf_best = best_model_rf.predict(X_rf)
accuracy_rf_best = accuracy_score(y_rf, predictions_rf_best)
confusion_matrix_rf_best = confusion_matrix(y_rf, predictions_rf_best)

precision_rf_best = precision_score(y_rf, predictions_rf_best, average='weighted')
recall_rf_best = recall_score(y_rf, predictions_rf_best, average='weighted')
f1_rf_best = f1_score(y_rf, predictions_rf_best, average='weighted')

predictions_proba_rf_best = best_model_rf.predict_proba(X_rf)

logloss_rf_best = log_loss(y_rf, predictions_proba_rf_best)

roc_auc_rf_best = roc_auc_score(y_rf, predictions_proba_rf_best, multi_class='ovr')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_rf_best, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza RandomForest RFC con i migliori parametri:", accuracy_rf_best)
print("Precisione RandomForest RFC con i migliori parametri:", precision_rf_best)
print("Recall RandomForest RFC con i migliori parametri:", recall_rf_best)
print("F1-score RandomForest RFC con i migliori parametri:", f1_rf_best)
print("Log-loss RandomForest RFC con i migliori parametri:", logloss_rf_best)
print("ROC-AUC RandomForest RFC con i migliori parametri:", roc_auc_rf_best)

print("Label nel training set:")
print(y_train_rf.value_counts())

print("\nLabel nel test set:")
print(y_test_rf.value_counts())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss, roc_auc_score

X_rf = new_dataset_rf.drop(' Label', axis=1)
y_rf = new_dataset_rf[' Label']

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

model_rf = RandomForestClassifier()

# Definizione della griglia dei parametri da cercare
param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_rf, param_grid, random_state=42).fit(X_train_rf, y_train_rf)
best_rf_params = search.best_params_

print("Migliori parametri per RandomForest:", best_rf_params)

best_model_rf = RandomForestClassifier(**best_rf_params)
best_model_rf.fit(X_train_rf, y_train_rf)

predictions_rf_best = best_model_rf.predict(X_test_rf)
accuracy_rf_best = accuracy_score(y_test_rf, predictions_rf_best)
confusion_matrix_rf_best = confusion_matrix(y_test_rf, predictions_rf_best)

precision_rf_best = precision_score(y_test_rf, predictions_rf_best, average='weighted')
recall_rf_best = recall_score(y_test_rf, predictions_rf_best, average='weighted')
f1_rf_best = f1_score(y_test_rf, predictions_rf_best, average='weighted')

predictions_proba_rf_best = best_model_rf.predict_proba(X_test_rf)

logloss_rf_best = log_loss(y_test_rf, predictions_proba_rf_best)

roc_auc_rf_best = roc_auc_score(y_test_rf, predictions_proba_rf_best, multi_class='ovr')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_rf_best, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza RandomForest RFC con i migliori parametri:", accuracy_rf_best)
print("Precisione RandomForest RFC con i migliori parametri:", precision_rf_best)
print("Recall RandomForest RFC con i migliori parametri:", recall_rf_best)
print("F1-score RandomForest RFC con i migliori parametri:", f1_rf_best)
print("Log-loss RandomForest RFC con i migliori parametri:", logloss_rf_best)
print("ROC-AUC RandomForest RFC con i migliori parametri:", roc_auc_rf_best)

print("Label nel training set:")
print(y_train_rf.value_counts())

print("\nLabel nel test set:")
print(y_test_rf.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import log_loss, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

X_svm = new_dataset_rf.drop(' Label', axis=1)
y_svm = new_dataset_rf[' Label']

X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X_svm, y_svm, test_size=None, random_state=42)

model_svm = SVC(probability=True)

# Definizione della griglia dei parametri da cercare
param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search_svm = HalvingGridSearchCV(model_svm, param_grid, cv=5, random_state=42).fit(X_svm, y_svm)
best_svm_params = search_svm.best_params_

print("Migliori parametri per Support Vector Machine:", best_svm_params)

best_model_svm = SVC(**best_svm_params, probability=True)
best_model_svm.fit(X_svm, y_svm)

predictions_svm_best = best_model_svm.predict(X_svm)
accuracy_svm_best = accuracy_score(y_svm, predictions_svm_best)
confusion_matrix_svm_best = confusion_matrix(y_svm, predictions_svm_best)
precision_svm_best = precision_score(y_svm, predictions_svm_best, average='weighted')
recall_svm_best = recall_score(y_svm, predictions_svm_best, average='weighted')
f1_svm_best = f1_score(y_svm, predictions_svm_best, average='weighted')

predictions_proba_svm_best = best_model_svm.predict_proba(X_svm)

logloss_svm_best = log_loss(y_svm, predictions_proba_svm_best)

roc_auc_svm_best = roc_auc_score(y_svm, predictions_proba_svm_best, multi_class='ovr')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_svm_best, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza Support Vector Machine con i migliori parametri:", accuracy_svm_best)
print("Precisione Support Vector Machine con i migliori parametri:", precision_svm_best)
print("Recall Support Vector Machine con i migliori parametri:", recall_svm_best)
print("F1-score Support Vector Machine con i migliori parametri:", f1_svm_best)
print("Log-loss Support Vector Machine con i migliori parametri:", logloss_svm_best)
print("ROC-AUC Support Vector Machine con i migliori parametri:", roc_auc_svm_best)

print("Label nel test set:")
print(y_test_svm.value_counts())


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss, roc_auc_score

X_svm_rf = new_dataset_rf.drop(' Label', axis=1)
y_svm_rf = new_dataset_rf[' Label']

X_train_svm_rf, X_test_svm_rf, y_train_svm_rf, y_test_svm_rf = train_test_split(X_svm_rf, y_svm_rf, test_size=None, random_state=42)

model_svm = SVC()

param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

# Applica Halving Grid Search CV
search_svm = HalvingGridSearchCV(model_svm, param_grid, cv=4, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state=42)
search_svm.fit(X_svm_rf, y_svm_rf)

best_svm_params = search_svm.best_params_

print("Migliori parametri per SVM:", best_svm_params)

best_model_svm_rf = SVC(**best_svm_params)
best_model_svm_rf.fit(X_svm_rf, y_svm_rf)

predictions_svm_rf = best_model_svm_rf.predict(X_svm_rf)
accuracy_svm_rf = accuracy_score(y_svm_rf, predictions_svm_rf)
confusion_matrix_svm_rf = confusion_matrix(y_svm_rf, predictions_svm_rf)
precision_svm_rf = precision_score(y_svm_rf, predictions_svm_rf, average='weighted')
recall_svm_rf = recall_score(y_svm_rf, predictions_svm_rf, average='weighted')
f1_svm_rf = f1_score(y_svm_rf, predictions_svm_rf, average='weighted')

predictions_proba_svm_rf = svm_calibrated.predict_proba(X_svm_rf)

logloss_svm_rf = log_loss(y_svm_rf, predictions_proba_svm_rf)

roc_auc_svm_rf = roc_auc_score(y_svm_rf, predictions_proba_svm_rf, multi_class='ovr')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_svm_rf, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza SVM RFC con i migliori parametri:", accuracy_svm_rf)
print("Precisione SVM RFC con i migliori parametri:", precision_svm_rf)
print("Recall SVM RFC con i migliori parametri:", recall_svm_rf)
print("F1-score SVM RFC con i migliori parametri:", f1_svm_rf)
print("Log-loss SVM RFC con i migliori parametri:", logloss_svm_rf)
print("ROC-AUC SVM RFC con i migliori parametri:", roc_auc_svm_rf)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, log_loss, roc_auc_score

X_rf = new_dataset_rf.drop(' Label', axis=1)
y_rf = new_dataset_rf[' Label']

model_rf = RandomForestClassifier()
model_rf.fit(X_rf, y_rf)

predictions_rf = model_rf.predict(X_rf)
accuracy_rf = accuracy_score(y_rf, predictions_rf)
confusion_matrix_rf = confusion_matrix(y_rf, predictions_rf)
precision_rf = precision_score(y_rf, predictions_rf, average='weighted')
recall_rf = recall_score(y_rf, predictions_rf, average='weighted')
f1_rf = f1_score(y_rf, predictions_rf, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_rf, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza RandomForest RFC:", accuracy_rf)
print("Precisione RandomForest RFC:", precision_rf)
print("Recall RandomForest RFC:", recall_rf)
print("F1-score RandomForest RFC:", f1_rf)

predictions_proba_rf = model_rf.predict_proba(X_rf)

logloss_rf = log_loss(y_rf, predictions_proba_rf)

roc_auc_rf = roc_auc_score(y_rf, predictions_proba_rf, average='weighted', multi_class='ovr')

print("Log-loss RandomForest RFC:", logloss_rf)
print("ROC-AUC RandomForest RFC:", roc_auc_rf)


In [ ]:
#Random Forest Test RFC

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

X_rf = new_dataset_rf.drop(' Label', axis=1)
y_rf = new_dataset_rf[' Label']

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

model_rf = RandomForestClassifier()
model_rf.fit(X_train_rf, y_train_rf)

predictions_rf = model_rf.predict(X_test_rf)
accuracy_rf = accuracy_score(y_test_rf, predictions_rf)
confusion_matrix_rf = confusion_matrix(y_test_rf, predictions_rf)
precision_rf = precision_score(y_test_rf, predictions_rf, average='weighted')
recall_rf = recall_score(y_test_rf, predictions_rf, average='weighted')
f1_rf = f1_score(y_test_rf, predictions_rf, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_rf, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza RandomForest RFC:", accuracy_rf)
print("Precisione RandomForest RFC:", precision_rf)
print("Recall RandomForest RFC:", recall_rf)
print("F1-score RandomForest RFC:", f1_rf)

from sklearn.metrics import log_loss, roc_auc_score

predictions_proba_rf = model_rf.predict_proba(X_test_rf)

logloss_rf = log_loss(y_test_rf, predictions_proba_rf)

roc_auc_rf = roc_auc_score(y_test_rf, predictions_proba_rf, average='weighted', multi_class='ovr')

print("Log-loss RandomForest RFC:", logloss_rf)
print("ROC-AUC RandomForest RFC:", roc_auc_rf)



In [ ]:
#Random Forest Test RFC attacchi

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X_rf = new_dataset_rf.drop(' Label', axis=1)
y_rf = new_dataset_rf[' Label']

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

model_rf = RandomForestClassifier()
model_rf.fit(X_train_rf, y_train_rf)

predictions_rf = model_rf.predict(X_test_rf)
accuracy_rf = accuracy_score(y_test_rf, predictions_rf)
confusion_matrix_rf = confusion_matrix(y_test_rf, predictions_rf)

print("Accuratezza RandomForest RFC:", accuracy_rf)
print("Matrice di confusione RandomForest RFC:\n", confusion_matrix_rf)

class_report_rf = classification_report(y_test_rf, predictions_rf)
print("Classification Report - Random Forest RFC:\n", class_report_rf)

class_labels_rf = model_rf.classes_

for i, label in enumerate(class_labels_rf):
    fn = confusion_matrix_rf[i, :].sum() - confusion_matrix_rf[i, i]
    fp = confusion_matrix_rf[:, i].sum() - confusion_matrix_rf[i, i]

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")


In [ ]:
#SVM test RFC attacchi


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X_svm_rf = new_dataset_rf.drop(' Label', axis=1)
y_svm_rf = new_dataset_rf[' Label']

model_rf = RandomForestClassifier()
model_rf.fit(X_svm_rf, y_svm_rf)

predictions_svm_rf = model_svm.predict(X_test_svm_rf)
accuracy_svm_rf = accuracy_score(y_test_svm_rf, predictions_svm_rf)
confusion_matrix_svm_rf = confusion_matrix(y_test_svm_rf, predictions_svm_rf)

svm_params = model_svm.get_params()

print("Iperparametri utilizzati nel modello SVM:")
for param, value in svm_params.items():
    print(f"{param}: {value}")

print("Accuratezza SVM RFC:", accuracy_svm_rf)
print("Matrice di Confusione SVM RFC:\n", confusion_matrix_svm_rf)

class_report_svm_rf = classification_report(y_test_svm_rf, predictions_svm_rf)
print("Classification Report - Random Forest RFC:\n", class_report_svm_rf)

class_labels_svm_rf = model_svm.classes_

for i, label in enumerate(class_labels_svm_rf):
    fn = confusion_matrix_svm_rf[i, :].sum() - confusion_matrix_svm_rf[i, i]
    fp = confusion_matrix_svm_rf[:, i].sum() - confusion_matrix_svm_rf[i, i]

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
#SVM Test RFC

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

X_svm_rf = new_dataset_rf.drop(' Label', axis=1)
y_svm_rf = new_dataset_rf[' Label']

model_svm = SVC()
model_svm.fit(X_svm_rf, y_svm_rf)

predictions_svm_rf = model_svm.predict(X_svm_rf)
accuracy_svm_rf = accuracy_score(y_svm_rf, predictions_svm_rf)
confusion_matrix_svm_rf = confusion_matrix(y_svm_rf, predictions_svm_rf)
precision_svm_rf = precision_score(y_svm_rf, predictions_svm_rf, average='weighted')
recall_svm_rf = recall_score(y_svm_rf, predictions_svm_rf, average='weighted')
f1_svm_rf = f1_score(y_svm_rf, predictions_svm_rf, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_svm_rf, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

svm_params = model_svm.get_params()

print("Iperparametri utilizzati nel modello SVM:")
for param, value in svm_params.items():
    print(f"{param}: {value}")

print("Accuratezza SVM RFC:", accuracy_svm_rf)
print("Precisione SVM RFC:", precision_svm_rf)
print("Richiamo SVM RFC:", recall_svm_rf)
print("F1-score SVM RFC:", f1_svm_rf)

from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss, roc_auc_score

calibrated_svm = CalibratedClassifierCV(model_svm, method='sigmoid', cv='prefit')
calibrated_svm.fit(X_svm_rf, y_svm_rf)

predictions_proba_svm_rf = calibrated_svm.predict_proba(X_svm_rf)

svm_log_loss = log_loss(y_svm_rf, predictions_proba_svm_rf)

svm_roc_auc = roc_auc_score(y_svm_rf, predictions_proba_svm_rf, average='weighted', multi_class='ovr')

print("Log-loss SVM RFC:", svm_log_loss)
print("ROC-AUC SVM RFC:", svm_roc_auc)



In [ ]:
!pip install boruta
import pandas as pd
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

X = dataset_subset[feature_cols]
Y = dataset_subset[' Label']
X_values = X.values
Y_values = Y.values

rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

boruta_selector = BorutaPy(rf, n_estimators= 50 , verbose=0, max_iter = 100)

np.int = np.int32
np.float = np.float64
np.bool = np.bool_

boruta_selector.fit(X_values, Y_values)

selected_features = X.columns[boruta_selector.support_]
ranks = boruta_selector.ranking_

print("Dimensione di selected_features:", len(selected_features))
print("Dimensione di boruta_selector.ranking_:", len(ranks))
print("Selected Features:", selected_features[:50])
print("Ranks:", boruta_selector.ranking_[:50])

X_selected = X[selected_features]

X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=None, random_state=42, stratify = Y)

rf_selected_features = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

rf_selected_features.fit(X.values, Y.values)

Y_pred = rf_selected_features.predict(X.values)

feature_importances = rf_selected_features.feature_importances_

feature_weights = {feature: importance for feature, importance in zip(selected_features, feature_importances)}

sorted_features = sorted(feature_weights, key=feature_weights.get, reverse=True)

weights_boruta = {feature: weight for feature, weight in zip(sorted_features, range(1, len(sorted_features) + 1))}

boruta_results = pd.DataFrame({'Feature': selected_features[:25], 'Rank': boruta_selector.ranking_[:25]})
boruta_results = boruta_results.sort_values(by='Rank', ascending=True)

plt.figure(figsize=(12, 6))
plt.bar(range(len(boruta_results)), boruta_results['Rank'], tick_label=boruta_results['Feature'], color = 'skyblue')
plt.xticks(rotation=90)
plt.title('Boruta')
plt.xlabel('Caratteristiche')
plt.ylabel('Rango')
plt.show()

print(boruta_results)

In [ ]:
# Seleziona le prime 10/20/30 feature da Boruta
top_features_boruta = boruta_results['Feature'][-30:][::-1].tolist()

selected_features_boruta = [' Label'] + top_features_boruta

new_dataset_boruta = dataset_subset[selected_features_boruta]

print(new_dataset_boruta.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

X_boruta = new_dataset_boruta.drop(' Label', axis=1)
y_boruta = new_dataset_boruta[' Label']

X_train_boruta, X_test_boruta, y_train_boruta, y_test_boruta = train_test_split(X_boruta, y_boruta, test_size=None, random_state=42)

model_boruta = RandomForestClassifier()

param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_boruta, param_grid, random_state=42).fit(X_boruta, y_boruta)
best_boruta_params = search.best_params_

print("Migliori parametri per RandomForestClassifier:", best_boruta_params)

best_model_boruta = RandomForestClassifier(**best_boruta_params)
best_model_boruta.fit(X_boruta, y_boruta)

predictions_boruta_best = best_model_boruta.predict(X_boruta)
accuracy_boruta_best = accuracy_score(y_boruta, predictions_boruta_best)
confusion_matrix_boruta_best = confusion_matrix(y_boruta, predictions_boruta_best)
precision_boruta_best = precision_score(y_boruta, predictions_boruta_best, average='weighted')
recall_boruta_best = recall_score(y_boruta, predictions_boruta_best, average='weighted')
f1_boruta_best = f1_score(y_boruta, predictions_boruta_best, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_boruta_best, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
print("Accuratezza RandomForest Boruta con i migliori parametri:", accuracy_boruta_best)
print("Precisione RandomForest RFC:", precision_boruta_best)
print("Recall RandomForest RFC:", recall_boruta_best)
print("F1-score RandomForest RFC:", f1_boruta_best)

from sklearn.metrics import log_loss, roc_auc_score

probabilities_boruta_best = best_model_boruta.predict_proba(X_boruta)

logloss_boruta_best = log_loss(y_boruta, probabilities_boruta_best)
print("Log-loss RandomForest Boruta con i migliori parametri:", logloss_boruta_best)

roc_auc_boruta_best = roc_auc_score(y_boruta, probabilities_boruta_best, multi_class='ovr')
print("Area sotto la curva ROC RandomForest Boruta con i migliori parametri:", roc_auc_boruta_best)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

X_svm_b = new_dataset_boruta.drop(' Label', axis=1)
y_svm_b = new_dataset_boruta[' Label']

X_train_svm_b, X_test_svm_b, y_train_svm_b, y_test_svm_b = train_test_split(X_svm_b, y_svm_b, test_size=None, random_state=42)

model_svm_b = SVC()

param_grid = {'C': [0.5], 'kernel': ['rbf'], 'tol': [0.001]}

search_svm = HalvingGridSearchCV(model_svm_b, param_grid, cv=5, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state = 42)
search_svm.fit(X_svm_b, y_svm_b)

best_svm_params = search_svm.best_params_

print("Migliori parametri per SVM:", best_svm_params)

best_model_svm_b = SVC(**best_svm_params, probability = True)
best_model_svm_b.fit(X_svm_b, y_svm_b)

predictions_svm_b = best_model_svm_b.predict(X_svm_b)
accuracy_svm_b = accuracy_score(y_svm_b, predictions_svm_b)
confusion_matrix_svm_b = confusion_matrix(y_svm_b, predictions_svm_b)
precision_svm_b = precision_score(y_svm_b, predictions_svm_b, average='weighted')
recall_svm_b = recall_score(y_svm_b, predictions_svm_b, average='weighted')
f1_svm_b = f1_score(y_svm_b, predictions_svm_b, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_svm_b, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
print("Accuratezza SVM Boruta con i migliori parametri:", accuracy_svm_b)
print("Precisione RandomForest RFC:", precision_svm_b)
print("Recall RandomForest RFC:", recall_svm_b)
print("F1-score RandomForest RFC:", f1_svm_b)

from sklearn.metrics import log_loss, roc_auc_score

probabilities_svm_b = best_model_svm_b.predict_proba(X_svm_b)

logloss_svm_b = log_loss(y_svm_b, probabilities_svm_b)
print("Log-loss SVM Boruta con i migliori parametri:", logloss_svm_b)

roc_auc_svm_b = roc_auc_score(y_svm_b, probabilities_svm_b, multi_class='ovr')
print("Area sotto la curva ROC SVM Boruta con i migliori parametri:", roc_auc_svm_b)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

X_boruta = new_dataset_boruta.drop(' Label', axis=1)
y_boruta = new_dataset_boruta[' Label']

X_train_boruta, X_test_boruta, y_train_boruta, y_test_boruta = train_test_split(X_boruta, y_boruta, test_size=None, random_state=42)

model_boruta = RandomForestClassifier()
model_boruta.fit(X_boruta, y_boruta)

predictions_boruta = model_boruta.predict(X_boruta)
accuracy_boruta = accuracy_score(y_boruta, predictions_boruta)
confusion_matrix_boruta = confusion_matrix(y_boruta, predictions_boruta)
precision_boruta = precision_score(y_boruta, predictions_boruta, average='weighted')
recall_boruta = recall_score(y_boruta, predictions_boruta, average='weighted')
f1_boruta = f1_score(y_boruta, predictions_boruta, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_boruta, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
print("Accuratezza RandomForest Boruta:", accuracy_boruta)
print("Precisione RandomForest RFC:", precision_boruta)
print("Recall RandomForest RFC:", recall_boruta)
print("F1-score RandomForest RFC:", f1_boruta)

from sklearn.metrics import log_loss, roc_auc_score

probabilities_boruta = model_boruta.predict_proba(X_boruta)

logloss_boruta = log_loss(y_boruta, probabilities_boruta)

roc_auc_boruta = roc_auc_score(y_boruta, probabilities_boruta, multi_class='ovr')

print("Log-loss RandomForest Boruta:", logloss_boruta)
print("ROC-AUC RandomForest Boruta:", roc_auc_boruta)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X_boruta = new_dataset_boruta.drop(' Label', axis=1)
y_boruta = new_dataset_boruta[' Label']

X_train_boruta, X_test_boruta, y_train_boruta, y_test_boruta = train_test_split(X_boruta, y_boruta, test_size=0.2, random_state=42)

model_boruta = RandomForestClassifier()
model_boruta.fit(X_train_boruta, y_train_boruta)

predictions_boruta = model_boruta.predict(X_test_boruta)
accuracy_boruta = accuracy_score(y_test_boruta, predictions_boruta)
confusion_matrix_boruta = confusion_matrix(y_test_boruta, predictions_boruta)

print("Accuratezza RandomForest Boruta:", accuracy_boruta)
print("Matrice di confusione RandomForest Boruta:\n", confusion_matrix_boruta)

class_report_b_rf = classification_report(y_test_boruta, predictions_boruta)
print("Classification Report - Random Forest RFC:\n", class_report_b_rf)

class_labels_b_rf = model_boruta.classes_

for i, label in enumerate(class_labels_b_rf):
    fn = confusion_matrix_boruta[i, :].sum() - confusion_matrix_boruta[i, i]
    fp = confusion_matrix_boruta[:, i].sum() - confusion_matrix_boruta[i, i]

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X_svm_b = new_dataset_boruta.drop(' Label', axis=1)
y_svm_b = new_dataset_boruta[' Label']

X_train_svm_b, X_test_svm_b, y_train_svm_b, y_test_svm_b = train_test_split(X_svm_b, y_svm_b, test_size=0.2, random_state=42)

model_svm_b = SVC()
model_svm_b.fit(X_train_svm_b, y_train_svm_b)

predictions_svm_b = model_svm_b.predict(X_test_svm_b)
accuracy_svm_b = accuracy_score(y_test_svm_b, predictions_svm_b)
confusion_matrix_svm_b = confusion_matrix(y_test_svm_b, predictions_svm_b)

print("Accuratezza SVM RFC:", accuracy_svm_b)
print("Matrice di Confusione SVM RFC:\n", confusion_matrix_svm_b)

class_report_svm_b = classification_report(y_test_svm_b, predictions_svm_b)
print("Classification Report - Random Forest RFC:\n", class_report_svm_b)

class_labels_svm_b = model_svm_b.classes_

for i, label in enumerate(class_labels_b_rf):
    fn = confusion_matrix_svm_b[i, :].sum() - confusion_matrix_svm_b[i, i]
    fp = confusion_matrix_svm_b[:, i].sum() - confusion_matrix_svm_b[i, i]

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X_svm_b = new_dataset_boruta.drop(' Label', axis=1)
y_svm_b = new_dataset_boruta[' Label']

X_train_svm_b, X_test_svm_b, y_train_svm_b, y_test_svm_b = train_test_split(X_svm_b, y_svm_b, test_size=None, random_state=42)

model_svm_b = SVC()
model_svm_b.fit(X_svm_b, y_svm_b)

predictions_svm_b = model_svm_b.predict(X_svm_b)
accuracy_svm_b = accuracy_score(y_svm_b, predictions_svm_b)
confusion_matrix_svm_b = confusion_matrix(y_svm_b, predictions_svm_b)
precision_svm_b = precision_score(y_svm_b, predictions_svm_b, average='weighted')
recall_svm_b = recall_score(y_svm_b, predictions_svm_b, average='weighted')
f1_svm_b = f1_score(y_svm_b, predictions_svm_b, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_svm_b, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza SVM RFC:", accuracy_svm_b)
print("Precisione RandomForest RFC:", precision_svm_b)
print("Recall RandomForest RFC:", recall_svm_b)
print("F1-score RandomForest RFC:", f1_svm_b)

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

decision_function_svm_b = model_svm_b.decision_function(X_svm_b)

label_binarizer = LabelBinarizer()
label_binarizer.fit(y_svm_b)
y_test_svm_b_bin = label_binarizer.transform(y_svm_b)

probabilities_svm_b = (decision_function_svm_b - decision_function_svm_b.min()) / (decision_function_svm_b.max() - decision_function_svm_b.min())

logloss_svm_b = log_loss(y_test_svm_b_bin, probabilities_svm_b)

roc_auc_svm_b = roc_auc_score(y_test_svm_b_bin, probabilities_svm_b, average='weighted')

print("Log-loss SVM Boruta:", logloss_svm_b)
print("ROC-AUC SVM Boruta:", roc_auc_svm_b)


In [ ]:
pca = PCA(n_components=50)
pca_res = pca.fit_transform(dataset_subset[feature_cols])

loadings = np.abs(pca.components_[0])

top_variables_indices = np.argsort(loadings)[::-1]

N = 50
top_variables = top_variables_indices[:N]

column_names = dataset_subset.columns

top_variable_names = [column_names[idx] for idx in top_variables]
top_variable_loadings = [loadings[idx] for idx in top_variables]

weights_pca = {column_names[idx]: weight for weight, idx in enumerate(top_variables_indices, start=1)}

plt.figure(figsize=(12,8))
plt.bar(top_variable_names, top_variable_loadings, color='skyblue')
plt.xlabel('Carichi delle Variabili')
plt.title('Carichi delle Variabili più Influenti dopo PCA')
plt.xticks(rotation=45, ha='right')
plt.show()

total_variance = np.sum(np.var(dataset_subset, axis=0))
explained_variance = np.sum(pca.explained_variance_[:N])
accuracy_pca = (explained_variance/total_variance)*100

for idx in top_variables:
    variable_name = column_names[idx]
    print(f"Variabile: {variable_name}, Carico: {loadings[idx]}")

print(accuracy_pca)

data_subset = None

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

features = dataset_subset.drop(' Label', axis=1)
target = dataset_subset[' Label']
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(pca_res[:, :30], target, test_size=None, random_state=42)

model_pca_rf = RandomForestClassifier()

param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

# Ricerca dei parametri migliori con Halving Grid Search CV
search = HalvingGridSearchCV(model_pca_rf, param_grid, random_state=42).fit(features, target)
best_pca_params = search.best_params_

print("Migliori parametri per RandomForestClassifier:", best_pca_params)

best_model_pca = RandomForestClassifier(**best_pca_params)
best_model_pca.fit(features, target)

predictions_pca_best = best_model_pca.predict(features)
accuracy_pca_best = accuracy_score(target, predictions_pca_best)
confusion_matrix_pca_best = confusion_matrix(target, predictions_pca_best)
precision_pca_best = precision_score(target, predictions_pca_best, average='weighted')
recall_pca_best = recall_score(target, predictions_pca_best, average='weighted')
f1_score_pca_best = f1_score(target, predictions_pca_best, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_pca_best, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza RandomForest PCA con i migliori parametri:", accuracy_pca_best)
print("Precisione RandomForest PCA con i migliori parametri:", precision_pca_best)
print("Recall RandomForest PCA con i migliori parametri:", recall_pca_best)
print("F1-score RandomForest PCA con i migliori parametri:", f1_score_pca_best)

from sklearn.metrics import log_loss, roc_auc_score

probabilities_pca_best = best_model_pca.predict_proba(features)

log_loss_pca_best = log_loss(target, probabilities_pca_best)

roc_auc_pca_best = roc_auc_score(target, probabilities_pca_best, multi_class='ovr', average='weighted')

print("Log-loss RandomForest PCA con i migliori parametri:", log_loss_pca_best)
print("ROC-AUC RandomForest PCA con i migliori parametri:", roc_auc_pca_best)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

features = dataset_subset.drop(' Label', axis=1)
target = dataset_subset[' Label']
X_train_svm_pca, X_test_svm_pca, y_train_svm_pca, y_test_svm_pca = train_test_split(pca_res[:, :30], target, test_size=None, random_state=42)

svm_model_pca = SVC()

param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

search_svm = HalvingGridSearchCV(svm_model_pca, param_grid, cv=5, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state=42)
search_svm.fit(features, target)

best_svm_params = search_svm.best_params_

print("Migliori parametri per SVM:", best_svm_params)

best_model_svm_pca = SVC(**best_svm_params)
best_model_svm_pca.fit(features, target)

predictions_svm_pca = best_model_svm_pca.predict(features)
accuracy_svm_pca = accuracy_score(target, predictions_svm_pca)
confusion_matrix_svm_pca = confusion_matrix(target, predictions_svm_pca)
precision_svm_pca = precision_score(target, predictions_svm_pca, average='weighted')
recall_svm_pca = recall_score(target, predictions_svm_pca, average='weighted')
f1_score_svm_pca = f1_score(target, predictions_svm_pca, average='weighted')

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_svm_pca, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print("Accuratezza SVM PCA con i migliori parametri:", accuracy_svm_pca)
print("Precisione SVM PCA con i migliori parametri:", precision_svm_pca)
print("Recall SVM PCA con i migliori parametri:", recall_svm_pca)
print("F1-score SVM PCA con i migliori parametri:", f1_score_svm_pca)

svm_model_ovr = OneVsRestClassifier(SVC(**best_svm_params, probability=True))
svm_model_ovr.fit(features, target)

svm_probabilities_ovr = svm_model_ovr.predict_proba(features)

logloss_svm_pca = log_loss(target, svm_probabilities_ovr)

y_test_svm_pca_binarized = label_binarize(target, classes=svm_model_ovr.classes_)

roc_auc_svm_pca = roc_auc_score(y_test_svm_pca_binarized, svm_probabilities_ovr, average='weighted')

print("Log-loss per SVM PCA:", logloss_svm_pca)
print("Area sotto la curva ROC (ROC-AUC) per SVM PCA:", roc_auc_svm_pca)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

features = dataset_subset.drop(' Label', axis=1)
target = dataset_subset[' Label']
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(pca_res[:, :30], target, test_size=None, random_state=42)

rf_model_pca = RandomForestClassifier()
rf_model_pca.fit(features, target)
rf_predictions_pca = rf_model_pca.predict(features)

rf_accuracy_pca = accuracy_score(target, rf_predictions_pca)
precision_rf_pca = precision_score(target, rf_predictions_pca, average='weighted')
recall_rf_pca = recall_score(target, rf_predictions_pca, average='weighted')
f1_score_rf_pca = f1_score(target, rf_predictions_pca, average='weighted')
print("Accuracy Random Forest:", rf_accuracy_pca)

rf_conf_matrix_pca = confusion_matrix(target, rf_predictions_pca)
plt.figure(figsize=(8, 6))
sns.heatmap(rf_conf_matrix_pca, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
print("Precisione Random Forest con PCA:", precision_rf_pca)
print("Recall Random Forest con PCA:", recall_rf_pca)
print("F1-score Random Forest con PCA:", f1_score_rf_pca)

from sklearn.metrics import log_loss, roc_auc_score

rf_probabilities_pca = rf_model_pca.predict_proba(features)

log_loss_rf_pca = log_loss(target, rf_probabilities_pca, labels=rf_model_pca.classes_)

roc_auc_rf_pca = roc_auc_score(target, rf_probabilities_pca, average='weighted', multi_class='ovr')

print("Log-loss Random Forest con PCA:", log_loss_rf_pca)
print("ROC-AUC Random Forest con PCA:", roc_auc_rf_pca)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

target = dataset_subset[' Label']
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(pca_res[:, :20], target, test_size=0.2, random_state=42)

model_pca_rf = RandomForestClassifier()

param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

search = HalvingGridSearchCV(model_pca_rf, param_grid, random_state=42).fit(X_train_pca, y_train_pca)
best_pca_params = search.best_params_

print("Migliori parametri per RandomForestClassifier:", best_pca_params)

best_model_pca = RandomForestClassifier(**best_pca_params)
best_model_pca.fit(X_train_pca, y_train_pca)

predictions_pca_best = best_model_pca.predict(X_test_pca)
accuracy_pca_best = accuracy_score(y_test_pca, predictions_pca_best)
confusion_matrix_pca_best = confusion_matrix(y_test_pca, predictions_pca_best)

print("Accuratezza RandomForest PCA con i migliori parametri:", accuracy_pca_best)
print("Matrice di confusione RandomForest PCA con i migliori parametri:\n", confusion_matrix_pca_best)

class_report_pca = classification_report(y_test_pca, predictions_pca_best)
print("Classification Report - Random Forest RFC:\n", class_report_pca)

class_labels_pca = best_model_pca.classes_

for i, label in enumerate(class_labels_pca):
    fn = confusion_matrix_pca_best[i, :].sum() - confusion_matrix_pca_best[i, i]
    fp = confusion_matrix_pca_best[:, i].sum() - confusion_matrix_pca_best[i, i]

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

target = dataset_subset[' Label']
X_train_svm_pca, X_test_svm_pca, y_train_svm_pca, y_test_svm_pca = train_test_split(pca_res[:, :10], target, test_size=0.2, random_state=42)

svm_model_pca = SVC()

param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

search_svm = HalvingGridSearchCV(svm_model_pca, param_grid, cv=5, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state = 42)
search_svm.fit(X_train_svm_pca, y_train_svm_pca)

best_svm_params = search_svm.best_params_

print("Migliori parametri per SVM:", best_svm_params)

best_model_svm_pca = SVC(**best_svm_params)
best_model_svm_pca.fit(X_train_svm_pca, y_train_svm_pca)

predictions_svm_pca = best_model_svm_pca.predict(X_test_svm_pca)
accuracy_svm_pca = accuracy_score(y_test_svm_pca, predictions_svm_pca)
confusion_matrix_svm_pca = confusion_matrix(y_test_svm_pca, predictions_svm_pca)

print("Accuratezza SVM PCA con i migliori parametri:", accuracy_svm_pca)
print("Matrice di confusione SVM PCA con i migliori parametri:\n", confusion_matrix_svm_pca)

class_report_svm_pca = classification_report(y_test_svm_pca, predictions_svm_pca)
print("Classification Report - Random Forest RFC:\n", class_report_svm_pca)

class_labels_svm_pca = best_model_svm_pca.classes_

for i, label in enumerate(class_labels_svm_pca):
    fn = confusion_matrix_svm_pca[i, :].sum() - confusion_matrix_svm_pca[i, i]
    fp = confusion_matrix_svm_pca[:, i].sum() - confusion_matrix_svm_pca[i, i]

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

features = dataset_subset.drop(' Label', axis=1)
target = dataset_subset[' Label']
X_train_svm_pca, X_test_svm_pca, y_train_svm_pca, y_test_svm_pca = train_test_split(pca_res[:, :30], target, test_size=0.2, random_state=42)

svm_model_pca = SVC()
svm_model_pca.fit(features, target)
svm_predictions_pca = svm_model_pca.predict(features)

svm_accuracy_pca = accuracy_score(target, svm_predictions_pca)
precision_svm_pca = precision_score(target, svm_predictions_pca, average='weighted')
recall_svm_pca = recall_score(target, svm_predictions_pca, average='weighted')
f1_score_svm_pca = f1_score(target, svm_predictions_pca, average='weighted')

print("Accuratezza SVM PCA:", svm_accuracy_pca)

svm_conf_matrix_pca = confusion_matrix(target, svm_predictions_pca)
plt.figure(figsize=(8, 6))
sns.heatmap(svm_conf_matrix_pca, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
print("Precisione SVM PCA:", precision_svm_pca)
print("Recall SVM PCA:", recall_svm_pca)
print("F1-score SVM PCA:", f1_score_svm_pca)

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

ovr_model = OneVsRestClassifier(SVC(probability=True))
ovr_model.fit(features, target)

svm_probabilities_pca = ovr_model.predict_proba(features)

log_loss_svm_pca = log_loss(target, svm_probabilities_pca)

roc_auc_svm_pca = roc_auc_score(target, svm_probabilities_pca, average='weighted', multi_class='ovr')

print("Log-loss SVM PCA:", log_loss_svm_pca)
print("ROC-AUC SVM PCA:", roc_auc_svm_pca)



In [ ]:
total_accuracy = accuracy_pca + accuracy_boruta + accuracy_rf
alpha_pca = accuracy_pca / total_accuracy
alpha_boruta = accuracy_boruta / total_accuracy
alpha_rf = accuracy_rf / total_accuracy

combined_ranking = {}

for feature in feature_df['Feature']:
    borda_score_pca = weights_pca.get(feature, 0)
    borda_score_boruta = weights_boruta.get(feature, 0)
    borda_score_rf = weights_rf.get(feature, 0)

    wborda_score_pca = weights_pca.get(feature, 0)
    wborda_score_boruta = weights_boruta.get(feature, 0)
    wborda_score_rf = weights_rf.get(feature, 0)

    combined_score = (
        alpha_pca * borda_score_pca + alpha_boruta * borda_score_boruta + alpha_rf * borda_score_rf +
        alpha_pca * wborda_score_pca + alpha_boruta * wborda_score_boruta + alpha_rf * wborda_score_rf
    )

    combined_ranking[feature] = combined_score

combined_ranking_sorted = {k: v for k, v in sorted(combined_ranking.items(), key=lambda item: item[1], reverse=True)}

for feature, score in combined_ranking_sorted.items():
    print(f"Feature: {feature}, Punteggio Combinato: {score:.4f}")


In [ ]:
# Seleziona le prime 10/20/30 feature dal ranking combinato
top_features_combined = list(combined_ranking_sorted.keys())[:20]

selected_features_combined = [' Label'] + top_features_combined

new_dataset_combined = dataset_subset[selected_features_combined]

print(new_dataset_combined.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

X_rf_combined = new_dataset_combined.drop(' Label', axis=1)
y_rf_combined = new_dataset_combined[' Label']

X_train_rf_combined, X_test_rf_combined, y_train_rf_combined, y_test_rf_combined = train_test_split(X_rf_combined, y_rf_combined, test_size=0.2, random_state=42)

model_rf_combined = RandomForestClassifier()

param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

search = HalvingGridSearchCV(model_rf_combined, param_grid, random_state=42).fit(X_train_rf_combined, y_train_rf_combined)
best_combined_params = search.best_params_

print("Migliori parametri per RandomForest:", best_combined_params)

best_model_combined = RandomForestClassifier(**best_combined_params)
best_model_combined.fit(X_train_rf_combined, y_train_rf_combined)

predictions_combined_best = best_model_combined.predict(X_test_rf_combined)
accuracy_combined_best = accuracy_score(y_test_rf_combined, predictions_combined_best)
confusion_matrix_combined_best = confusion_matrix(y_test_rf_combined, predictions_combined_best)

print("Accuratezza RandomForest R.combinato con i migliori parametri:", accuracy_combined_best)
print("Matrice di confusione RandomForest R.combinato con i migliori parametri:\n", confusion_matrix_combined_best)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

X_svm_combined = new_dataset_combined.drop(' Label', axis=1)
y_svm_combined = new_dataset_combined[' Label']

X_train_svm_combined, X_test_svm_combined, y_train_svm_combined, y_test_svm_combined = train_test_split(X_svm_combined, y_svm_combined, test_size=0.2, random_state=42)

model_svm_combined = SVC()

param_grid = {'C': [0.1, 0.5, 1, 2, 5, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'tol': [1e-3, 1e-2]}

search_svm = HalvingGridSearchCV(model_svm_combined, param_grid, cv=5, factor=2, max_resources=110, min_resources=10, aggressive_elimination=False, random_state = 42)
search_svm.fit(X_train_svm_combined, y_train_svm_combined)

best_svm_params = search_svm.best_params_

print("Migliori parametri per SVM:", best_svm_params)

best_model_svm_combined = SVC(**best_svm_params)
best_model_svm_combined.fit(X_train_svm_combined, y_train_svm_combined)

predictions_svm_combined = best_model_svm_combined.predict(X_test_svm_combined)
accuracy_svm_combined = accuracy_score(y_test_svm_combined, predictions_svm_combined)
confusion_matrix_svm_combined = confusion_matrix(y_test_svm_combined, predictions_svm_combined)

print("Accuratezza SVM R.combinato con i migliori parametri:", accuracy_svm_combined)
print("Matrice di confusione SVM R.combinato con i migliori parametri:\n", confusion_matrix_svm_combined)

In [ ]:
#RF per ranking combinato
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

X_rf_combined = new_dataset_combined.drop(' Label', axis=1)
y_rf_combined = new_dataset_combined[' Label']

X_train_rf_combined, X_test_rf_combined, y_train_rf_combined, y_test_rf_combined = train_test_split(X_rf_combined, y_rf_combined, test_size=0.2, random_state=42)

model_rf_combined = RandomForestClassifier()
model_rf_combined.fit(X_train_rf_combined, y_train_rf_combined)

predictions_rf_combined = model_rf_combined.predict(X_test_rf_combined)
accuracy_rf_combined = accuracy_score(y_test_rf_combined, predictions_rf_combined)
confusion_matrix_rf_combined = confusion_matrix(y_test_rf_combined, predictions_rf_combined)

print("Accuratezza RandomForest Ranking Combinato:", accuracy_rf_combined)
print("Matrice Confusione RandomForest Ranking Combinato:\n", confusion_matrix_rf_combined)


In [ ]:
#RF per ranking combinato
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X_rf_combined = new_dataset_combined.drop(' Label', axis=1)
y_rf_combined = new_dataset_combined[' Label']

X_train_rf_combined, X_test_rf_combined, y_train_rf_combined, y_test_rf_combined = train_test_split(X_rf_combined, y_rf_combined, test_size=0.2, random_state=42)

model_rf_combined = RandomForestClassifier()
model_rf_combined.fit(X_train_rf_combined, y_train_rf_combined)

predictions_rf_combined = model_rf_combined.predict(X_test_rf_combined)
accuracy_rf_combined = accuracy_score(y_test_rf_combined, predictions_rf_combined)
confusion_matrix_rf_combined = confusion_matrix(y_test_rf_combined, predictions_rf_combined)

print("Accuratezza RandomForest Ranking Combinato:", accuracy_rf_combined)
print("Matrice Confusione RandomForest Ranking Combinato:\n", confusion_matrix_rf_combined)

class_report_combined = classification_report(y_test_rf_combined, predictions_rf_combined)
print("Classification Report - Random Forest RFC:\n", class_report_combined)

class_labels_combined = model_rf_combined.classes_

for i, label in enumerate(class_labels_combined):
    fn = confusion_matrix_rf_combined[i, :].sum() - confusion_matrix_rf_combined[i, i]
    fp = confusion_matrix_rf_combined[:, i].sum() - confusion_matrix_rf_combined[i, i]

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
#SVM per ranking combinato
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X_svm_combined = new_dataset_combined.drop(' Label', axis=1)
y_svm_combined = new_dataset_combined[' Label']

X_train_svm_combined, X_test_svm_combined, y_train_svm_combined, y_test_svm_combined = train_test_split(X_svm_combined, y_svm_combined, test_size=0.2, random_state=42)

model_svm_combined = SVC()
model_svm_combined.fit(X_train_svm_combined, y_train_svm_combined)

predictions_svm_combined = model_svm_combined.predict(X_test_svm_combined)
accuracy_svm_combined = accuracy_score(y_test_svm_combined, predictions_svm_combined)
confusion_matrix_svm_combined = confusion_matrix(y_test_svm_combined, predictions_svm_combined)

print("Accuracy SVM (con le prime 20 feature):", accuracy_svm_combined)
print("Confusion Matrix SVM (con le prime 20 feature):\n", confusion_matrix_svm_combined)

class_report_svm_combined = classification_report(y_test_svm_combined, predictions_svm_combined)
print("Classification Report - Random Forest RFC:\n", class_report_svm_combined)

class_labels_svm_combined = model_svm_combined.classes_

for i, label in enumerate(class_labels_svm_combined):
    fn = confusion_matrix_svm_combined[i, :].sum() - confusion_matrix_svm_combined[i, i]
    fp = confusion_matrix_svm_combined[:, i].sum() - confusion_matrix_svm_combined[i, i]

    print(f"\nClasse: {label}")
    print(f"False Negative (FN): {fn}")
    print(f"False Positive (FP): {fp}")

In [ ]:
#SVM per ranking combinato
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X_svm_combined = new_dataset_combined.drop(' Label', axis=1)
y_svm_combined = new_dataset_combined[' Label']

X_train_svm_combined, X_test_svm_combined, y_train_svm_combined, y_test_svm_combined = train_test_split(X_svm_combined, y_svm_combined, test_size=0.2, random_state=42)

model_svm_combined = SVC()
model_svm_combined.fit(X_train_svm_combined, y_train_svm_combined)

predictions_svm_combined = model_svm_combined.predict(X_test_svm_combined)
accuracy_svm_combined = accuracy_score(y_test_svm_combined, predictions_svm_combined)
confusion_matrix_svm_combined = confusion_matrix(y_test_svm_combined, predictions_svm_combined)

print("Accuracy SVM (con le prime 20 feature):", accuracy_svm_combined)
print("Confusion Matrix SVM (con le prime 20 feature):\n", confusion_matrix_svm_combined)


In [ ]:
tsne = TSNE(n_components=2, verbose=0, perplexity=100, n_iter=1000)
tsne_res = tsne.fit_transform(pca_res)
print("done")

In [ ]:
dataset_subset['tsne_firstD'] = tsne_res[:,0]
dataset_subset['tsne_secondD'] = tsne_res[:,1]
dataset_subset

In [ ]:
custom_palette = [
    (0, 0, 0),       # Nero
    (1, 1, 0),       # Giallo
    (1, 0, 0),       # Rosso
    (0, 1, 0),       # Verde
    (1, 0.549, 0),   # Arancione
    (1, 0, 1),       # Fuchsia
    (0, 1, 1),       # Ciano
    (0, 0, 1),       # Blu
    (0.502, 0, 0.502),  # Lilla
    (0.502, 0.502, 0),  # Olive
    (0, 0.749, 1)   # Turchese
]

In [ ]:
plt.figure(figsize=(16,16))

sns.scatterplot(
    x="tsne_firstD",
    y="tsne_secondD",
    palette=custom_palette,
    data=dataset_subset,
    hue=' Label',
    legend="full",
    alpha=0.75
)

plt.xlabel('tsne_firstD')
plt.ylabel('tsne_secondD')
plt.title('Anomaly Detection')
plt.show()